[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **LLMs Vs AI Agents**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

Welcome to this hands-on notebook! In this walkthrough, we'll explore two essential building blocks of agentic AI using **local models**:

1. **Simple LLM Inference**
   Learn how to make a direct call to a local Large Language Model (LLM) using Ollama and generate a response.

2. **Making Your First AI Agent**
   Understand how an LLM can be extended with memory, tools, and behaviors using SmolAgents framework.

This notebook is part of the *Build your AI Agents* series — designed to break down complex concepts into simple, practical code examples using **local models** that run on your machine.

**Prerequisites:**
- Ollama installed and running (`ollama serve`)
- Models downloaded (e.g., `ollama pull qwen2.5vl:7b`)

![SmolAgents Logo](https://camo.githubusercontent.com/c6efa99360afde7cf829dff3cad81e56573658c1843464dff1fbb30a8f63b082/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f68756767696e67666163652f646f63756d656e746174696f6e2d696d616765732f7265736f6c76652f6d61696e2f736d6f6c6167656e74732f736d6f6c6167656e74732e706e67)

## Part 1 – Simple Local LLM Inference with Ollama

Large language models (LLMs) are predictive text models trained on vast amounts of data. They generate responses by predicting the next word given a prompt. This section shows how to make a direct call to a **local LLM** running on Ollama.

**Benefits of Local LLMs:**
- 🔒 **Privacy** - Your data never leaves your machine
- 💰 **Cost** - No API charges, unlimited usage
- ⚡ **Speed** - No network latency
- 🎛️ **Control** - Full control over model parameters

**Note:** Make sure Ollama is running (`ollama serve`) and you have models available (`ollama list`).

In [2]:
# 📦 Install required packages
!pip install requests -q

import requests
import json

# 🔍 Check if Ollama is running and list available models
def check_ollama_status():
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=5)
        if response.status_code == 200:
            models = response.json().get('models', [])
            print("✅ Ollama is running!")
            print(f"📦 Available models: {len(models)}")
            
            # Show available models
            for i, model in enumerate(models[:5]):
                print(f"   {i+1}. {model['name']}")
            return [model['name'] for model in models]
        else:
            print("❌ Ollama not responding")
            return []
    except Exception as e:
        print(f"❌ Cannot connect to Ollama: {e}")
        print("   Please start Ollama: ollama serve")
        return []

available_models = check_ollama_status()

✅ Ollama is running!
📦 Available models: 4
   1. qwen3:4b
   2. qwen2.5vl:7b
   3. llama3.2-vision:11b
   4. llama3.2-vision:latest


In [ ]:
# ✨ Simple LLM call using Ollama API
def call_ollama_model(prompt, model="qwen3:8b"):
    """Make a direct call to Ollama model"""
    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post(url, json=data, timeout=60)
        if response.status_code == 200:
            return response.json()['response']
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Error calling Ollama: {str(e)}"

# Test simple LLM inference
prompt = "What is an AI agent? Explain in simple terms."
response = call_ollama_model(prompt, "qwen2.5vl:7b")

print("📝 Simple LLM Response:")
print(response)

In this code, we connect directly to Ollama's API endpoint, send a prompt to `qwen3:8b` (or your preferred model), and get back a single response.

Simple LLM calls like this are **reactive**: they respond to the prompt but do not remember previous interactions or take any action beyond producing text.

**Key differences from cloud APIs:**
- No API keys required
- Models run locally on your hardware
- Response time depends on your machine's capabilities
- Complete privacy and control over data

## Part 2 – Building an AI Agent with SmolAgents

You can create agents in many ways — either **without a framework** (by wiring together an LLM with memory, tools, and control loops manually) or **with a framework** that handles these components for you.

Here, we'll use **SmolAgents** from Hugging Face, which is designed to be:
- 🚀 **Simple** - Only ~1,000 lines of core code
- 🧑‍💻 **Code-first** - Agents write Python code to solve problems
- 🔧 **Model-agnostic** - Works with any LLM (local or cloud)
- 🛠️ **Tool-friendly** - Easy integration with external tools

SmolAgents turns a plain LLM into an **agent** by augmenting it with:
- 🧠 **Memory** – keeps track of interactions through conversation history
- 🛠️ **Tools** – lets the agent execute Python code and use external APIs
- 🤔 **Reasoning** – allows step-by-step problem solving with code

**Why SmolAgents is perfect for local models:**
- Works great with Ollama models
- Lightweight and fast
- No external dependencies for basic functionality
- Code execution happens locally

Let's install SmolAgents and create our first agent!

In [4]:
# 📦 Install SmolAgents with toolkit
!pip install "smolagents[toolkit]" -q
!pip install openai -q


from smolagents import CodeAgent, OpenAIServerModel

print("✅ SmolAgents installed!")

# 🤖 Create Ollama model for SmolAgents
# SmolAgents uses OpenAI-compatible API, which Ollama provides
model = OpenAIServerModel(
    model_id="qwen3:4b",  # Your local model
    api_base="http://localhost:11434/v1",  # Ollama's OpenAI-compatible endpoint
    api_key="ollama",  # Ollama doesn't need a real API key
    temperature=0.1  # Lower temperature for more consistent responses
)

print("🤖 Local model configured for SmolAgents!")
print(f"   Model: qwen3:4b")
print(f"   Endpoint: http://localhost:11434/v1")
print(f"   Running locally via Ollama")

✅ SmolAgents installed!
🤖 Local model configured for SmolAgents!
   Model: qwen3:4b
   Endpoint: http://localhost:11434/v1
   Running locally via Ollama


In [5]:
# 🎯 Create a basic SmolAgent
from smolagents import DuckDuckGoSearchTool
    
agent_with_search = CodeAgent(
    tools=[DuckDuckGoSearchTool()],  # Now web search is available
    model=model,
    instructions="You are a helpful assistant. You can search the web when needed to provide accurate information."
)

print("🔍 Agent with web search created!")
question = "What are the main differences between a pure LLM and an AI agent?"
response = agent_with_search.run(question)
print("\n🌐 Agent with Search Response:")
print(response)
    

🔍 Agent with web search created!


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What are the main differences between a pure LLM and an AI agent?                                               │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen3:4b ──────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = web_search(query="main differences between pure LLM and AI agent")                                      
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[AI Agents Vs LLMs: Understanding The Differences & ...](https://acecloud.ai/blog/ai-agents-vs-llms/)
Jul 3, 2025 — Unlike LLMs, AI agents don't generate text – they execute tasks. They are designed to work 
autonomously, making decisions based on real world ...

[Understanding the Differences Between LLM, RAG, and AI 
...](https://medium.com/@hannanorris591/understanding-the-differences-between-llm-rag-and-ai-agents-for-businesses-
1196da15c513)
Unlike LLMs that generate answers or RAG systems that provide accurate insights, AI Agents combine intelligence 
with action . They can understand ...

[Unpacking the Power of AI Agents vs. Pure 
LLMs](https://www.dsstream.com/post/beyond-text-generation-unpacking-the-power-of-ai-agents-vs-pure-llms)
The shift from pure LLMs to AI agents represents a pivotal step in AI evolution. Both have their place, but they 
serve different purposes.

[LLMs to AI Agents: What's the Difference and Why It 
Matters](https://digitalworkforce.com/rpa-news/live-webinar-from-llms-to-ai-agents-whats-the-difference-and-why-it-
matters/)
Why enterprise-grade agents are more reliable, stable, and auditable than pure LLM solutions. The role of 
orchestration, tool use, and context in real-world ...

[Beyond Text Generation: Understanding the Power of AI Agents 
...](https://medium.com/@ds_stream/beyond-text-generation-understanding-the-power-of-ai-agents-vs-pure-llms-0ec8981
f80b1)
AI agents, by contrast, build upon LLMs but extend far beyond text generation. They maintain state across 
interactions, integrate with external tools and APIs, ...

[AI Agent vs LLM (Large Language Model)](https://bito.ai/blog/ai-agent-vs-llm/)
Mar 13, 2025 — The key distinction remains: if the AI is just generating content (no matter how impressive), it's 
acting as an LLM ; if it's deciding and doing ...

[AI Agents vs. AI Assistants](https://www.ibm.com/think/topics/ai-agents-vs-ai-assistants)
Decision-making and action: The ability to call on tools by itself does not make an LLM an agent . AI agents can 
also act autonomously and decide which tools to ...

[If you're unsure what Agentic AI is and what's the difference 
...](https://www.reddit.com/r/AI_Agents/comments/1hqzzrg/if_youre_unsure_what_agentic_ai_is_and_whats_the/)
The key distinction between AI agents and workflows actually lies in the autonomy of decision-making. AI agents can
independently determine ...

[LLM → RAG → AI Workflow → AI 
Agent](https://www.codelink.io/blog/post/ai-system-development-llm-rag-ai-workflow-agent)
A pure LLM is essentially a lossy compression of the internet, a snapshot of knowledge from its training data. It 
excels at tasks involving this stored ...

[Why and How to Build AI Agents for LLM Applications 
-](https://deepsense.ai/blog/why-and-how-to-build-ai-agents-for-llm-applications/)
Differences Between AI Copilots, Chatbots, and Autonomous Agents , and Decision Recommendation Systems ... and 
limitations becomes easier after ...

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The main differences between a pure LLM and an AI agent are: 1. Pure LLMs generate text            
  responses, while AI agents execute tasks autonomously. 2. AI agents maintain state across interactions, whereas  
  LLMs lack memory between sessions. 3. AI agents integrate with external tools and APIs, while LLMs are limited   
  to their training data.")                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The main differences between a pure LLM and an AI agent are: 1. Pure LLMs generate text responses, 
while AI agents execute tasks autonomously. 2. AI agents maintain state across interactions, whereas LLMs lack 
memory between sessions. 3. AI agents integrate with external tools and APIs, while LLMs are limited to their 
training data.

[Step 4: Duration 41.40 seconds| Input tokens: 12,270 | Output tokens: 3,655]


🌐 Agent with Search Response:
The main differences between a pure LLM and an AI agent are: 1. Pure LLMs generate text responses, while AI agents execute tasks autonomously. 2. AI agents maintain state across interactions, whereas LLMs lack memory between sessions. 3. AI agents integrate with external tools and APIs, while LLMs are limited to their training data.
